In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

In [2]:
dropout = pd.read_csv("02 cleaning_mv.csv")
dropout.shape

(2750, 188)

In [3]:
#variabili non presenti per casi 1.2 e 2.2
dropout.drop(['eta_pensiero_abbandono', 'stato_animo_pre_allenamento', 'no_divertimento', 'no_bravura', 'troppa_competizione', 
           'infortuni', 'eccessiva_fatica', 'incomprensioni_allenatore', 'motivi_salute', 'periodo_critico_personale', 
           'troppo_impegnativo', 'fattore_disabilita', 'no_appoggio_genitori', 'no_accompagnamento_genitori', 
           'rapporto_difficile_genitori_allenatore', 'difficolta_compagni', 'nessuno_con_cui_allenarsi', 'lontananza', 
           'male_scuola', 'no_conciliare_scuola_sport', 'altri_interessi', 'difficolta_raggiungere_impianto', 'sport_costoso', 
           'no_palestra_adeguata', 'materiale_carente', 'orari_scomodi', 'assenza_sport_vicino_casa', 'all_severo', 
           'all_autorevole', 'all_amico', 'all_competente', 'all_leader', 'all_fissato', 'all_riferimento', 'all_non_in_grado', 
           'all_avvilente', 'all_irritabile', 'all_motivatore', 'all_no_preparato', 'frequenza_scuola_giornaliera', 'integrazione_societa_sportiva',
           'attenzioni_allenatore', 'incoraggiamento_allenatore', 'stimolo_compagni', 'problema_comportamento_compagni',
           'allenamento_con_preoccupazioni', 'allenamento_con_stanchezza', 'allenamento_essendo_indaffarato', 
           'partecipazione_gare', 'sentimento_sconfitta', 'sentimento_vittoria'], 1, inplace=True)

#variabili da eliminare
dropout.drop(['Unnamed: 0', 'caso', 'respondent_id', 'anno_nascita', 'regione',
              'motivazione_principale_per_continuare', 'motivazione_principale_per_abbandonare',
              'flag_lavoro_madre', 'flag_lavoro_padre', 'ore_attivita_extra'], 1, inplace=True)

dropout.shape

(2750, 127)

In [4]:
categoriali = ['sesso', 'scuola_superiore', 'titolo_studio_madre', 'titolo_studio_padre', 'riferimento_dopo_scuola', 
               'provenienza', 'nazionalita', 'lavoro_madre', 'impegno_lavoro_madre', 'lavoro_padre', 'impegno_lavoro_padre',
               'influenza_scelta_sport', 'def_genitore', 'tipo_sport_significativo']

In [5]:
maschio_calcio = []
femmina_pallavolo_nuoto = []

for i in range(2750):
    if (dropout.sesso[i]=="Maschio" and dropout.sport_piu_significativo[i]=="Calcio"):
        maschio_calcio.append("Si")
    else:
        maschio_calcio.append("No")
    if (dropout.sesso[i]=="Femmina" and (dropout.sport_piu_significativo[i]=="Pallavolo" or dropout.sport_piu_significativo[i]=="Nuoto")):
        femmina_pallavolo_nuoto.append("Si")
    else:
        femmina_pallavolo_nuoto.append("No")

dropout['maschio_calcio'] = pd.Series(maschio_calcio)
dropout['femmina_pallavolo_nuoto'] = pd.Series(femmina_pallavolo_nuoto)

dropout.drop('sport_piu_significativo', 1, inplace=True)
dropout.shape

(2750, 128)

In [6]:
for i in range(2750):
    if (dropout.grado_divertimento_allenamento[i]=="Per niente"):
        dropout.grado_divertimento_allenamento[i] = 0
    if (dropout.grado_divertimento_allenamento[i]=="Poco"):
        dropout.grado_divertimento_allenamento[i] = 1
    if (dropout.grado_divertimento_allenamento[i]=="Neutrale"):
        dropout.grado_divertimento_allenamento[i] = 2
    if (dropout.grado_divertimento_allenamento[i]=="Abbastanza"):
        dropout.grado_divertimento_allenamento[i] = 3
    if (dropout.grado_divertimento_allenamento[i]=="Molto"):
        dropout.grado_divertimento_allenamento[i] = 4
        
dropout.grado_divertimento_allenamento.head()

0    3
1    3
2    3
3    0
4    2
Name: grado_divertimento_allenamento, dtype: object

In [7]:
distanze = ['tempo_casa_struttura_sportiva_piedi', 'tempo_casa_struttura_sportiva_mezzi', 
            'tempo_scuola_struttura_sportiva_piedi', 'tempo_scuola_struttura_sportiva_mezzi']

for var in distanze:
    for i in range(2750):
        if (dropout[var][i]=="Meno di un quarto d'ora"):
            dropout[var][i] = 0
        if (dropout[var][i]=="Un quarto d'ora"):
            dropout[var][i] = 1
        if (dropout[var][i]=="Più di un quarto d'ora"):
            dropout[var][i] = 2
        if (dropout[var][i]=="Un'ora"):
            dropout[var][i] = 3
        if (dropout[var][i]=="Più di un'ora"):
            dropout[var][i] = 4

dropout.tempo_casa_struttura_sportiva_piedi.head()

0    0
1    0
2    2
3    3
4    0
Name: tempo_casa_struttura_sportiva_piedi, dtype: object

In [8]:
from sklearn.preprocessing import LabelEncoder

for var in dropout.columns:
    if var not in categoriali:
        enc = LabelEncoder()
        dropout[var] = enc.fit_transform(dropout[var])
        
dropout.shape

(2750, 128)

In [9]:
#One Hot Encoding - Custom
for c in categoriali:
    vals = dropout[c].unique()
    vals.sort()
    for val in vals:
        column = c + "_" + val.lower().replace(" ", "_")
        l = []
        for i in range(2750):
            if (dropout[c][i]==val):
                l.append(1)
            else:
                l.append(0)
        dropout[column] = pd.Series(l)
    dropout.drop(c, 1, inplace=True)
    
dropout.shape

(2750, 186)

In [10]:
for var in dropout.columns:
    dropout[var] = dropout[var].astype(int)

In [11]:
dropout.head()

,disabilita,numero_fratelli_sorelle,numero_nonni_vicinanza_casa,cambio_residenza,presenza_campioni_sportivi_citta,arrampicata,arti_marziali,atletica,balli_da_sala,baseball,...,def_genitore_fastidioso,def_genitore_incoraggiante,def_genitore_indifferente,def_genitore_pesante,def_genitore_si_arrabbia_alle_gare,def_genitore_sostenitore_positivo,def_genitore_tranquillo,def_genitore_troppo_fissato,tipo_sport_significativo_individuale,tipo_sport_significativo_squadra
0,0,2,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,0,1,2,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
3,0,1,4,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,0,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [12]:
dropout.to_csv("03 encoding.csv")